# After the introductive slides on Xspec... let's try to interactively work with PyXspec now, starting from this template...

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
from astropy.table import  Table
def print_model(m):
    _comp = []
    _name = []
    _val = []
    _unit = []
    _err = []
    _step = []
    _min = []
    _max = []
    _froz = []
    colnames = ['component', 'par name', 'value', 'units', 'error', 'step', 'min', 'max', 'frozen']
    for model_name in m.componentNames:
        fit_model = getattr(m, model_name)
        for name in fit_model.parameterNames:
            p = getattr(fit_model, name)
            _comp.append('%s' % (model_name))
            _name.append('%s' % (p.name))
            _val.append('%5.4f' % p.values[0])
            _unit.append('%s' % p.unit)
            _err.append('%5.4f' % p.sigma)
            _step.append('%5.4f' % p.values[1])
            _min.append('%5.2e' % p.values[2])
            _max.append('%5.2e' % p.values[4])
            _froz.append('%s' % p.frozen)

    t=Table([_comp, _name, _val, _unit, _err, _step, _min, _max, _froz],names=colnames)
    print(t)

In [ ]:
import shutil
from IPython.display import Image
from IPython.display import display 
def plot_spectrum(what = "euf del", units = "keV", minSig=0, maxBins=1, command = ""):
    '''
    Possible unit formats are: "channel", "MeV", "Hz", "angstrom"
    rebin: minSig, maxBins, groupNum, errType 
    '''
#    xsp.Plot.addCommand("setplot delete all")
#    xsp.Plot.commands = ()
    xsp.Plot.device="/png"
    xsp.Plot.xAxis=units
    xsp.Plot.background = True
    xsp.Plot.setRebin(minSig=minSig, maxBins=maxBins)#, groupNum=1, errType="quad") #DOES NOT WORK
    if command:
        xsp.Plot.addCommand(command)    
    xsp.Plot(what)

    #xspec.Plot.addCommand("setplot en")
#    xsp.Plot.xLog = False
#    xsp.Plot.yLog = False
#    xsp.Plot("ufspec")
#    xsp.Plot("data")
#    xsp.Plot("model")
#    xsp.Plot("data chisq")
#    xsp.Plot("data","model","resid")

#    xsp.Plot.setRebin(minSig=10, maxBins=30, groupNum=1, errType="quad") #DOES NOT WORK
#    if bool(rebin):
#        print rebin
#        xsp.Plot.addCommand("setPlot rebin %s" % (rebin))
#    else:
#        print "no rebin"
#        xsp.Plot.addCommand("setplot delete all")
#        xsp.Plot.commands = ()
#    print xsp.Plot.commands

    xsp.Plot.device="/png"

    fn="test.png"


    shutil.move("pgplot.png_2", fn)

    _=display(Image(filename=fn,format="png"))
    
    return xsp.Plot

In [ ]:
def get_stats():
    chi2_red=xsp.Fit.statistic/xsp.Fit.dof
    chi2=xsp.Fit.statistic
    ndof=xsp.Fit.dof
    
    return chi2_red, chi2, ndof

In [ ]:
from future import *
import pprint
import xspec as xsp

In [ ]:
xsp.Xset.allowPrompting 
xsp.AllModels.clear()
xsp.AllData.clear()
xsp.AllChains.clear()

# Load the spectrum file
s = xsp.Spectrum('data/spec.fits')
# Load the response file (Energy vs. channels)
s.response = 'data/rmf.fits'
# Load the arf file (Effective Area vs. Energy)
s.response.arf = 'data/arf.fits'

In [ ]:
s.notice('all')
plot_spectrum(what='ldata', units='ch')
s.noticed

In [ ]:
xsp.AllData.ignore('bad')
s.noticed

In [ ]:
plot_spectrum(what='ldata', units='ch')

In [ ]:
s.ignore('**-2')
s.ignore('60-**')

In [ ]:
plot_spectrum(what='ldata', units='ch')
s.noticed

In [ ]:
plot_spectrum(what='ldata', units='keV')

In [ ]:
s.ignore('**-20.')
s.ignore('300.-**')

In [ ]:
plot_spectrum(what='ldata', units='keV')

In [ ]:
s.notice('all')
xsp.AllData.ignore('bad')
s.noticed

In [ ]:
# Define a model
m = xsp.Model('wabs*po')

In [ ]:
print_model(m)

In [ ]:
# Component objects are accessible-by-name as Model object attributes:
comp1 = m.wabs
comp2 = m.powerlaw
# Parameter objects are accessible-by-name as Component object attributes:
par1 = m.wabs.nH
par2 = m.powerlaw.PhoIndex
# ...and we can modify their values:
par2.values = 2.0
m.wabs.nH = 1.5
comp2.norm = 3.0

# Can also get a Parameter object directly from a Model, without going
# through a Component.  Just pass the Model the Parameter index number:
par3 = m(3)

In [ ]:
print_model(m)

In [ ]:
# Or you can use the 'setPars' method to change the value of a parameter
# model.setPars({nPar:value})
m.setPars({2:1.7}) # 

In [ ]:
print_model(m)

In [ ]:
# If you want to change e.g. the 'step' or 'boundary' you must use a list of 6 floats: [value, fit delta, min, bot, top, max]
m.powerlaw.PhoIndex.values = [1.0, 0.01, -4.0, -3.0, 6.0, 7.0]

In [ ]:
print_model(m)

In [ ]:
# To free / freeze a parameter:
par2.frozen = False
par3.frozen = True

In [ ]:
print_model(m)

In [ ]:
par1.link = "3.3 * 2" # Link par 1 to par 2 with a multiplicative constant 3.3
par1.link = "" # Removes the link.
par1.untie()   # Also removes the link.

In [ ]:
print_model(m)

In [ ]:
# Clear the model to its original value
m = xsp.Model('wabs*po')

In [ ]:
xsp.Fit.renorm()

In [ ]:
xsp.Fit.query = 'yes'
xsp.Fit.perform()

In [ ]:
print_model(m)

In [ ]:
chi2_red, chi2, ndof =get_stats()
print('chi_red',chi2_red)
print('chi',chi2)
print('ndof',ndof)

In [ ]:
plot_spectrum(what='ldata', units='keV')

In [ ]:
print_model(m)

In [ ]:
# Step parameter 3 from 1.5 to 2.5 in 10 linear steps
xsp.Fit.steppar("2 1.7 3.7 10")
step2 = xsp.Fit.stepparResults('delstat')
xsp.Fit.steppar("3 0.6 60. 10")
step3 = xsp.Fit.stepparResults('delstat')

# Repeat the above but with logarithmic steps
#xsp.Fit.steppar("log")
# Step parameter 2 linearly from -.2 to .2 in steps of .02
#xsp.Fit.steppar("nolog 2 -.2 .2 20")

In [ ]:
step2

In [ ]:
step3

In [ ]:
# Create and open a log file for XSPEC output
# This returns a Python file object
logFile = xsp.Xset.openLog("newLogFile.txt")

# commands with desired output:
xsp.Fit.show()
m.show()

# Get the Python file object for the currently opened log
logFile = xsp.Xset.log
# Close XSPEC's currently opened log file.
xsp.Xset.closeLog()

In [ ]:
less newLogFile.txt

In [ ]:
plot_spectrum(what='eeuf delchi', units='keV')

In [ ]:
plot_spectrum(what='eeuf delchi', units='keV', minSig=2, maxBins=10)

In [ ]:
# 'AllModels' is a container for all models
# 'AllData' is a container for all data
xsp.AllModels.calcFlux("50. 100.")
s1 = xsp.AllData(1)
s1.flux

In [ ]:
# Luminosity(enMin, enMax, redshift)
xsp.AllModels.calcLumin("50. 100. .05")
s1 = xsp.AllData(1)
s1.lumin

In [ ]:
#Estimate the 90% confidence range for the 2nd parameter
xsp.Fit.error("2.706 2")
par2 = xsp.AllModels(1)(2)
par2.error

In [ ]:
#Estimate the 90% confidence range for the 2nd parameter
xsp.Fit.error("1. 2")
par2 = xsp.AllModels(1)(2)
par2.error

# Load another spectra (NGC 7582)

In [ ]:
cd data

In [ ]:
xsp.AllModels.clear()
xsp.AllData.clear()
xsp.AllChains.clear()

s = xsp.Spectrum('nu_ngc7582_sr-30.pha')
#s.response = 'data/rmf.fits'
#s.response.arf = 'data/arf.fits'

#s.ignore('**-15')
#s.ignore('300-**')
xsp.AllData.ignore('bad')

# Try to fit first with a simple powlaw
m = xsp.Model('po')

In [ ]:
print_model(m)

In [ ]:
xsp.Fit.query = 'yes'
xsp.Fit.perform()

In [ ]:
print_model(m)

In [ ]:
chi2_red, chi2, ndof = get_stats()
print('chi_red',chi2_red)
print('chi',chi2)
print('ndof',ndof)

In [ ]:
plot_spectrum("euf del chisq")

In [ ]:
xsp.AllModels.clear()
# Add then the absorption
m = xsp.Model('wabs*po')

 List of Xspec models:
 https://heasarc.gsfc.nasa.gov/xanadu/xspec/manual/Models.html

In [ ]:
print_model(m)

In [ ]:
m.componentNames

In [ ]:
# Component objects are accessible-by-name as Model object attributes:
comp1 = m.wabs
comp2 = m.powerlaw

In [ ]:
comp1.parameterNames

In [ ]:
comp2.parameterNames

In [ ]:
# Parameter objects are accessible-by-name as Component object attributes:
par3 = comp2.norm

In [ ]:
# value, fit delta, min, bot, top, max
par3.values

In [ ]:
# ...and we can modify their values:
par3.values = 0.003
m.wabs.nH = 20
comp2.PhoIndex = 1.5

In [ ]:
print_model(m)

In [ ]:
plot_spectrum()

In [ ]:
xsp.Fit.query = 'yes'
xsp.Fit.perform()

chi2_red, chi2, ndof = get_stats()
print('chi_red',chi2_red)
print('chi',chi2)
print('ndof',ndof)

In [ ]:
print_model(m)

In [ ]:
plot_spectrum()

In [ ]:
xsp.AllModels.clear()
# Use a more appropriate model
m = xsp.Model('zphabs*(pexrav)')

In [ ]:
print_model(m)

In [ ]:
plot_spectrum()

In [ ]:
xsp.Fit.renorm()

In [ ]:
plot_spectrum()

In [ ]:
print_model(m)

In [ ]:
xsp.Fit.query = 'yes'
xsp.Fit.perform()

chi2_red, chi2, ndof = get_stats()
print('chi_red',chi2_red)
print('chi',chi2)
print('ndof',ndof)

In [ ]:
print_model(m)

In [ ]:
p = plot_spectrum()

In [ ]:
xVals = p.x()
yVals = p.y()
#yVals2 = p.y(2)  # Gets values for data in the second plot group
modVals = p.model()
# Retrieve error arrays
xErrs = p.xErr()
yErrs = p.yErr()

In [ ]:
yErrs

In [ ]:
plot_spectrum("euf del chisq")

In [ ]:
# Finally you can use an even more complex model
# m3 = xsp.Model('zphabs*(pexrav+zgauss)')

# Load another spectra if you want to try on your own... (NGC 5548)

In [ ]:
xsp.AllModels.clear()
xsp.AllData.clear()
xsp.AllChains.clear()

s = xsp.Spectrum("nu_ngc5548_obs1_sr-30.pha")

In [ ]:
...